In [26]:
import pandas as pd
import numpy as np
from numpy_financial import irr

In [4]:
payment_cols = ('LOAN_ID', 'IssuedDate', 'RECEIVED_D', 'RECEIVED_AMT_INVESTORS', 'PBAL_END_PERIOD_INVESTORS')
df_payments_1 = pd.read_csv('data/Payments_Made_to_Investors_File1of2_2020_08.csv', low_memory=False,  nrows=None, usecols=payment_cols)
df_payments_1.head()

,LOAN_ID,RECEIVED_D,RECEIVED_AMT_INVESTORS,PBAL_END_PERIOD_INVESTORS,IssuedDate
0,54734,SEP2009,632.771017,18636.409300,AUG2009
1,54734,OCT2009,632.771017,18188.363925,AUG2009
2,54734,NOV2009,632.771017,17735.877487,AUG2009
3,54734,DEC2009,632.771017,17278.905966,AUG2009
4,54734,JAN2010,632.771017,16817.404904,AUG2009


In [5]:
df_payments_2 = pd.read_csv('data/Payments_Made_to_Investors_File2of2_2020_08.csv', low_memory=False, skiprows=1, header=None, nrows=None, usecols=[0, 1, 11, 12, 16])
df_payments_2.head()

,0,1,11,12,16
0,90721450,JUN2018,37.94,571.313680,OCT2016
1,90721450,JUL2018,37.94,537.463334,OCT2016
2,90721450,AUG2018,37.94,503.370676,OCT2016
3,90721450,SEP2018,37.94,469.033971,OCT2016
4,90721450,OCT2018,37.94,434.451472,OCT2016


In [6]:
df_payments_2.columns = df_payments_1.columns
df_combined = pd.concat([df_payments_1, df_payments_2])

In [7]:
df_combined.head()

,LOAN_ID,RECEIVED_D,RECEIVED_AMT_INVESTORS,PBAL_END_PERIOD_INVESTORS,IssuedDate
0,54734,SEP2009,632.771017,18636.409300,AUG2009
1,54734,OCT2009,632.771017,18188.363925,AUG2009
2,54734,NOV2009,632.771017,17735.877487,AUG2009
3,54734,DEC2009,632.771017,17278.905966,AUG2009
4,54734,JAN2010,632.771017,16817.404904,AUG2009


In [8]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61820100 entries, 0 to 28703915
Data columns (total 5 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   LOAN_ID                    int64  
 1   RECEIVED_D                 object 
 2   RECEIVED_AMT_INVESTORS     float64
 3   PBAL_END_PERIOD_INVESTORS  float64
 4   IssuedDate                 object 
dtypes: float64(2), int64(1), object(2)
memory usage: 2.8+ GB


In [2]:
df = pd.read_pickle('data/df_payments_cleaned_training.pkl.bz2', compression='bz2')

In [3]:
df.to_pickle('data/df_payments_training_loans.pkl.bz2', compression='bz2', protocol=4)

In [1]:
def get_one_loan_payment_data(df_payments, loan_id):
    '''
    Function to extract payments made by a single loan ID. 

    Args:
        payments_training_loans (dataframe): The dataframe containing all loan payments data for our training loans.
            Only training loans are relevant since ROI needs to be calculated as our label to use in model training.
        loan_id (int): The loan ID that we want to get payments for.

    Returns:
        DataFrame: Returns a dataframe containing payment history for a single loan.

    Todo: Add in description of the format the payments_training_loans dataframe should be in.
    '''
    try:
        # Loan ID must be passed in as a list to ensure we get a dataframe back and not a series.
        # Otherwise a series is returned when we have a loan where only 1 payment has been made.
        return df_payments.loc[pd.IndexSlice[:, loan_id], ['RECEIVED_AMT_INVESTORS', 'mths_since_issue']]
    except:
        # Need to return an empty dataframe if no payments were found for the given loan_id.
        return pd.DataFrame()

def convert_monthly_return_to_annual(irr):
    return (1 + irr)**12 - 1    

def get_roi_for_loan_id(loan_id):
    starting_loan_balance = loan_amounts[loan_id]
    loan_payments = get_one_loan_payment_data(df_payments, loan_id)
    if len(loan_payments) == 0:
        return -100
    max_months = loan_payments['mths_since_issue'].max()
    payments = np.zeros(max_months+1)
    payments[0] = -starting_loan_balance
    for payment, month in zip(loan_payments['RECEIVED_AMT_INVESTORS'], loan_payments['mths_since_issue']):
        payments[month] += payment
    irr_monthly = irr(payments)
    irr_annual = convert_monthly_return_to_annual(irr_monthly)
    return 100 * irr_annual

In [42]:
import pickle 

with open('data/loan_amounts.pickle', 'rb') as handle:
    loan_amounts = pickle.load(handle)
    
with open('data/training_loan_ids.pickle', 'rb') as handle:
    training_loan_ids = pickle.load(handle)
    
with open('data/loan_rois.pickle', 'rb') as handle:
    loan_rois = pickle.load(handle)

In [6]:
len(training_loan_ids)

1097123

In [10]:
get_one_loan_payment_data(df_payments, 88082393)

,,RECEIVED_AMT_INVESTORS,mths_since_issue
RECEIVED_D,LOAN_ID,,
2016-08-01,88082393,15020.290039,0


In [80]:
df_payments = pd.read_pickle('data/df_payments_cleaned_all.pkl.bz2', compression='bz2')
df_payments.head()

RECEIVED_AMT_INVESTORS  PBAL_END_PERIOD_INVESTORS  \
RECEIVED_D LOAN_ID                                                      
2007-07-01 72176                  7.189307                 219.559830   
           73582                  7.289357                 219.637436   
           74505                  7.256250                 219.611313   
           77792                  3.975833                 121.962997   
           81085                  9.032310                 268.539795   

                   IssuedDate  mths_since_issue  
RECEIVED_D LOAN_ID                               
2007-07-01 72176   2007-06-01                 1  
           73582   2007-06-01                 1  
           74505   2007-06-01                 1  
           77792   2007-06-01                 1  
           81085   2007-06-01                 1

In [71]:
loan_ids_need_updating = set(df_payments[df_payments['mths_since_issue'] == 0].index.get_level_values(1))
len(loan_ids_need_updating)

39346

In [20]:
for loan_id in training_loan_ids:
    if loan_id not in loan_rois:
        loan_ids_need_updating.add(loan_id)

In [30]:
get_roi_for_loan_id(88082393)

1.6353338544679241

In [74]:
current_updates = [loan for loan in loan_ids_need_updating if loan not in needs_updating]
len(current_updates)

36080

In [81]:
len(current_updates)

36080

In [83]:
for loan_id in current_updates:
    if loan_id in loan_amounts:
        loan_rois[loan_id] = get_roi_for_loan_id(loan_id)

In [84]:
with open('data/loan_rois_updated.pickle', 'wb') as handle:
    pickle.dump(loan_rois, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [77]:
print('Now update the loans not originally in the ROI dictionary.')

Now update the loans not originally in the ROI dictionary.


In [67]:
needs_updating = []
for k, v in loan_rois.items():
    if math.isnan(v):
        needs_updating.append(k)
len(needs_updating)

3266

In [78]:
still_needed = [loan for loan in training_loan_ids if loan not in loan_rois and loan in loan_amounts]
len(still_needed)

24415

In [82]:
len(still_needed)

24415

In [85]:
for loan_id in still_needed:
    if loan_id in loan_amounts:
        loan_rois[loan_id] = get_roi_for_loan_id(loan_id)

In [86]:
with open('data/loan_rois_updated.pickle', 'wb') as handle:
    pickle.dump(loan_rois, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
training_loan_ids